# Ground state of Diatomic Hydrogen using XACC and VQE

In [ ]:
import numpy as np
import pyxacc as xacc
import pyxaccvqe as vqe
from openfermion.hamiltonians import MolecularData
from openfermionpsi4 import run_psi4
from openfermion.transforms import get_fermion_operator

## Define the Molecule

In [ ]:
bond_length = .7474
geometry = [('H', (0,0,0)),('H',(0,0,bond_length))]
basis = 'sto-3g'
mdata = MolecularData(geometry,basis,1)

## Generate the Hamiltonian coefficients and get FCI energy

In [ ]:
molecule = run_psi4(mdata, run_scf=1, run_fci=1)
print('FCI Energy = ', molecule.fci_energy)

## Map to OpenFermion FermionOperator

In [ ]:
fop = get_fermion_operator(molecule.get_molecular_hamiltonian()) 
print(fop)

## Compile to XACC Hamiltonian for Accelerator Execution

In [ ]:
xaccOp = vqe.compile(fop)
print(xaccOp)

## Execute VQE with Nelder Mead Optimizer and UCCSD Ansatz

In [ ]:
result = vqe.execute(xaccOp, **{'n-electrons':2,'task':'vqe'})
print('VQE Energy = ', result.energy)
print('VQE Optimized Angles = ', result.angles)

## Use Scipy to do VQE Optimization

In [ ]:
from scipy.optimize import minimize

def energy(p):
    return vqe.execute(xaccOp, **{'n-electrons':2, 
                                  'task':'compute-energy',
                                  'vqe-params':str(p[1])+','+str(p[1])}).energy

initial_angles = np.random.uniform(low=-np.pi, high=np.pi, size=(2,))

opt_result = minimize(energy, initial_angles,
                      method="nelder-mead", tol=1e-3, options={'disp':True})

opt_energy, opt_amplitudes = opt_result.fun, opt_result.x
print("\nOptimalVQE Energy: {}".format(opt_energy))
print("Optimal Angles: {}".format(opt_amplitudes))


## Exercise - Using above code, loop over bond_length, construct dissociation curve